<a href="https://colab.research.google.com/github/vinicius-souza-lima/mvp_ADBP/blob/main/MVP_ADBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requisitos

## Requisitos e composição da nota:

*   (1,0 pt) Execução sem erros:o notebook deve poder ser executado pelo professor do início ao fim sem erros.
*   (2,0 pts) Documentação consistente:utilize blocos de texto que expliquem textualmente cada etapa e cada decisão do seu código, contando uma história completa e compreensível, do início ao fim.
*   (1,0 pt) Código limpo:seu código deve estar legível e organizado. Devem ser utilizadas as boas práticas de codificação vistas nas disciplinas Programação orientada a objetose Engenharia de software para ciência de dados, mas não é necessário que você crie classes no seu código.
*   (2,0 pts) Análise de dados:após cada gráfico, você deverá escrever um parágrafo resumindo os principais achados, analisando os resultados e levantando eventuais pontos de atenção.
*   (2,0 pts) Checklist:você deverá responder às perguntas (aplicáveis ao seu dataset) da checklist fornecida, utilizando-a como guia para o desenvolvimento do trabalho.
*   (2,0 pts) Capricho e qualidade do trabalho como um todo.







## Definição do problema
Objetivo: entender e descrever claramente o problema que está sendo resolvido.

Qual é a descrição do problema?

Dado um conjunto de características do meu jogo,quero prever como será seu resultado na Steam.

Este é um problema de aprendizado supervisionado ou não supervisionado?

Supervisionado


Que premissas ou hipóteses você tem sobre o problema?
Que restrições ou condições foram impostas para selecionar os dados?
Defina cada um dos atributos do dataset.

## Análise de dados
Objetivo: entender a informação disponível.

Estatísticas descritivas:

Quantos atributos e instâncias existem?
Quais são os tipos de dados dos atributos?
Verifique as primeiras linhas do dataset. Algo chama a atenção?
Há valores faltantes, discrepantes ou inconsistentes?
Faça um resumo estatístico dos atributos com valor numérico (mínimo, máximo, mediana, moda, média, desvio-padrão e número de valores ausentes). O que você percebe?

## Visualizações:

Verifique a distribuição de cada atributo. O que você percebe? Dica: esta etapa pode dar ideias sobre a necessidade de transformações na etapa de preparação de dados (por exemplo, converter atributos de um tipo para outro, realizar operações de discretização, normalização, padronização, etc.).
Se for um problema de classificação, verifique a distribuição de frequência das classes. O que você percebe? Dica: esta etapa pode indicar a possível necessidade futura de balanceamento de classes.
Analise os atributos individualmente ou de forma combinada, usando os gráficos mais apropriados.

## Pré-processamento de dados:
Objetivo: realizar operações de limpeza, tratamento e preparação dos dados.

Verifique quais operações de pré-processamento podem ser interessantes para o seu problema e salve visões diferentes do seu dataset (por exemplo, normalização, padronização, discretização e one-hot-encoding).
Trate (removendo ou substituindo) os valores faltantes (se existentes).
Realize outras transformações de dados porventura necessárias.
Explique, passo a passo, as operações realizadas, justificando cada uma delas.
Se julgar necessário, utilizando os dados pré-processados, volte na etapa de análise exploratória e verifique se surge algum insight diferente após as operações realizadas.

# Introdução


Nome:Vinícius de Souza Lima.

Link para o Reposítório: [MVP Análise de Dados e Boas Práticas](https://github.com/vinicius-souza-lima/mvp_ADBP)


Foi utilizado um dataset proveniente do Kaggle, [Steam Games Dataset](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset?resource=download), o qual traz informações de mais de 85.000 jogos publicados na Steam, a maior plataforma de jogos de PC do mercado.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Upload dos dados( Dataset proveniente do )
games = pd.DataFrame({})
for i in range(10):
  url_dados = 'https://raw.githubusercontent.com/vinicius-souza-lima/mvp_ADBP/main/dataset/games-'+ str(i+1) + '.csv'
# carga do dataset através da url
  games = pd.concat([games,pd.read_csv(url_dados)])


In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85103 entries, 0 to 8494
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85103 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85103 non-null  object 
 3   Estimated owners            85103 non-null  object 
 4   Peak CCU                    85103 non-null  int64  
 5   Required age                85103 non-null  int64  
 6   Price                       85103 non-null  float64
 7   DLC count                   85103 non-null  int64  
 8   About the game              81536 non-null  object 
 9   Supported languages         85103 non-null  object 
 10  Full audio languages        85103 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85103 non-null  object 
 13  Website                     39452 non

In [ ]:
# Dicionário de Dados
colunas = games.columns
descriptions ="""Identificador Único,string
Nome do Jogo,string
Data de Lançamento,string
Estimativa de Possuidores,string
Número de Usuários Atuais,int
Classificação indicativa,int
Preço em Dólares Americanos,float
Número de DLCs,int
Descrição detalhada dos jogos,string
Descrição curta dos jogos,string
Enumeração separada por vírgula das línguas suportadas, lista de strings
Enumeração separada por vírgula das línguas com suporte de aúdio, lista de strings
Imagem Principal na Loja,string
Website do Jogo,string
URL do Suporte do Jogo,string
Suporta Windows?,bool
Suporta Mac?,bool
Suporta Linux?,bool
Score do Metacritic,int
URL da review do Metecritic,string
Score do usuário,int
Votos Positivos,int
Votos negativos,int
Rank do score baseado nas reviews de usuários,string
Número de Conquistas,int
Recomendações de usuários,int
Informações extras,string
Tempo de jogo médio desde março de 2009 em minutos,int
Average playtime in the last two weeks, in minutes (int).
Median playtime since March 2009, in minutes (int).
Median playtime in the last two weeks, in minutes (int).
Available packages.
Package title (string).
Package description (string).
Subpackages.
Subpackage title (string).
Subpackage description (string).
Subpackage price in USD (float).
Game developers.
Developer name (string).
Game publishers.
Publisher name (string).
Game categories.
Category name (string).
Game genres.
Gender name (string).
Game screenshots.
Game screenshot URL,string.
Game movies.
Game movie URL,string.
Tags.
Tag key (string, int).
"""

# Definição do Problema

# Análise de Dados

# Pré-Processamento de Dados

In [ ]:
# exibindo as últimas linhas
games.describe()

,AppID,Peak CCU,Required age,Price,DLC count,Metacritic score,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks
count,8.510300e+04,85103.000000,85103.000000,85103.000000,85103.000000,85103.000000,85103.000000,8.510300e+04,85103.000000,44.000000,85103.000000,8.510300e+04,85103.000000,85103.000000,85103.000000,85103.000000
mean,1.355681e+06,134.872930,0.312774,7.193703,0.543412,3.348366,0.039822,9.585609e+02,159.772570,98.909091,19.859394,7.755176e+02,104.729681,10.680105,93.316029,11.467328
std,6.949952e+05,5403.548851,2.254721,12.362478,13.721223,15.421471,1.791013,2.435920e+04,4574.583903,0.857747,171.446874,1.789338e+04,1142.447517,188.840006,1510.732101,205.372944
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,97.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,7.723900e+05,0.000000,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,98.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.331540e+06,0.000000,0.000000,4.490000,0.000000,0.000000,0.000000,7.000000e+00,2.000000,99.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
75%,1.918415e+06,1.000000,0.000000,9.990000,0.000000,0.000000,0.000000,4.500000e+01,14.000000,100.000000,18.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
max,2.765800e+06,872138.000000,21.000000,999.980000,2366.000000,97.000000,100.000000,5.764420e+06,895978.000000,100.000000,9821.000000,3.441592e+06,145727.000000,19159.000000,208473.000000,19159.000000


In [ ]:
games.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [ ]:
games["Required age"].value_counts(ascending=False)

,count
Required age,
0,83463
17,919
18,333
13,204
16,68
10,35
12,34
3,8
15,8


In [ ]:
games["Price"].value_counts(ascending=False)

,count
Price,
0.00,16461
4.99,7752
9.99,7199
0.99,6497
1.99,5192
...,...
4.40,1
7.94,1
39.39,1


In [ ]:
games["Positive"].value_counts(ascending=False)

,count
Positive,
0,23314
1,5291
2,3951
3,3051
4,2620
...,...
2399,1
21087,1
9544,1


# Conclusão